In [104]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

mnist = fetch_openml("mnist_784")

X, y = mnist.data.values, mnist.target.values

X_binary = (X > 128).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X_binary, y, test_size=0.2, random_state=42)

class BernoulliNBWithLog:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_priors = np.array([np.log(np.mean(y == c)) for c in self.classes])

        self.feature_probs = []
        for c in self.classes:
            feature_prob = (X[y == c].sum(axis=0)) / (np.sum(y == c))
            self.feature_probs.append(np.log(np.clip(feature_prob, 1e-10, 1.0 - 1e-10)))
        self.feature_probs = np.array(self.feature_probs)

    def predict(self, X):
        log_likelihoods = np.dot(X, self.feature_probs.T)
        log_posteriors = log_likelihoods + self.class_priors

        predicted_class = self.classes[np.argmax(log_posteriors, axis=1)]
        return predicted_class

model = BernoulliNBWithLog()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))



c:\Users\정효정\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Accuracy: 70.36%
